# Gallery Example: Multi-Class Closed Queueing Network

This example demonstrates a multi-class closed queueing network:
- **Classes**: Multiple job classes (default 2)
- **Stations**: Configurable number of queues (default 1)
- **Topology**: Serial routing with optional delay station
- **Population**: 5 jobs per class
- **Scheduling**: Processor Sharing (PS)
- **Service**: Random exponential service rates per class/station

This model is useful for analyzing systems with different customer types or job characteristics.

In [ ]:
from line_solver import *
import numpy as np
import random
GlobalConstants.set_verbose(VerboseLevel.STD)

In [ ]:
def gallery_cqn_multiclass(m=1, r=2, wantdelay=True, seed=None):    """Create multi-class closed queueing network        Parameters:    - m: Number of queue stations (default 1)    - r: Number of classes (default 2)    - wantdelay: Whether to include delay station (default True)    - seed: Random seed for reproducibility    """    if seed is not None:        random.seed(seed)        np.random.seed(seed)        model = Network('Multi-class CQN')        # Block 1: nodes    node = []    for i in range(m):        queue = Queue(model, f'Queue {i+1}', SchedStrategy.PS)        node.append(queue)        if wantdelay:        delay = Delay(model, 'Delay 1')        node.append(delay)        # Block 2: classes    jobclass = []    for s in range(r):        jclass = ClosedClass(model, f'Class{s+1}', 5, node[0], 0)        jobclass.append(jclass)        # Set service rates    service_rates = {}  # Store for analysis    for s in range(r):        service_rates[f'Class{s+1}'] = {}        for i in range(m):            rate = round(50 * random.random())  # Random rate 0-50            if rate == 0:                rate = 1  # Avoid zero service rate            node[i].set_service(jobclass[s], Exp.fit_mean(rate))            service_rates[f'Class{s+1}'][f'Queue{i+1}'] = rate                if wantdelay:            delay_rate = round(100 * random.random())  # Random delay 0-100            if delay_rate == 0:                delay_rate = 1            node[-1].set_service(jobclass[s], Exp.fit_mean(delay_rate))            service_rates[f'Class{s+1}']['Delay1'] = delay_rate        # Block 3: topology - serial routing for each class    P = model.init_routing_matrix()    for s in range(r):        # Create cyclic routing for this class        for i in range(len(node)):            next_node = node[(i + 1) % len(node)]            P.add_route(jobclass[s], node[i], next_node, 1.0)        model.link(P)        return model, service_rates# Create the model with default parametersmodel, service_rates = gallery_cqn_multiclass(m=1, r=2, wantdelay=True, seed=42)print(f"Population: 5 jobs per class")print(f"\nService rates (mean service times):")for class_name, rates in service_rates.items():    print(f"  {class_name}: {rates}")

## Multi-Class Network Analysis

In multi-class closed networks:
- **Classes compete** for server resources
- **Different service requirements** per class/station
- **Population constraints** limit total jobs per class
- **PS scheduling** provides fair sharing among classes

Performance depends on:
- Population mix and sizes
- Service rate differences between classes
- Network topology and routing

In [ ]:
# Solve with multiple solvers
print("\n=== Solver Results ===")

# MVA Solver (exact for closed networks)
solver_mva = MVA(model)
avg_table_mva = solver_mva.avg_table()
print("\nMVA Solver:")
print(avg_table_mva)

# CTMC Solver
solver_ctmc = CTMC(model)
avg_table_ctmc = solver_ctmc.avg_table()
print("\nCTMC Solver:")
print(avg_table_ctmc)

# NC Solver
solver_nc = NC(model)
avg_table_nc = solver_nc.avg_table()
print("\nNC Solver:")
print(avg_table_nc)

In [ ]:
# Analyze class-specific performance
print("\n=== Class-Specific Performance Analysis ===")

# Get detailed performance metrics by class
solver = MVA(model)
avg_table = solver.avg_table()

print("Per-class throughput and response times:")
for class_idx in range(model.get_number_of_classes()):
    class_name = f"Class{class_idx + 1}"
    print(f"\n{class_name}:")
    
    # Calculate total response time and throughput for this class
    total_resp_time = 0
    for station_idx in range(1, model.get_number_of_stations()):
        try:
            # Note: Table structure may vary - this is a general approach
            resp_time = float(avg_table.iloc[station_idx, 2])  # Response time column
            total_resp_time += resp_time
        except:
            pass
    
    print(f"  Service rates: {service_rates[class_name]}")
    print(f"  Population: 5 jobs")

In [ ]:
# Compare different class population mixes
print("\n=== Population Mix Analysis ===")

def create_cqn_with_populations(populations):
    """Create CQN with specified populations per class"""
    model_pop = Network('CQN-PopMix')
    
    # Create nodes
    queue = Queue(model_pop, 'Queue1', SchedStrategy.PS)
    delay = Delay(model_pop, 'Delay1')
    nodes = [queue, delay]
    
    # Create classes with different populations
    jobclasses = []
    for i, pop in enumerate(populations):
        jclass = ClosedClass(model_pop, f'Class{i+1}', pop, queue, 0)
        jobclasses.append(jclass)
        
        # Set service rates (fixed for comparison)
        queue.set_service(jclass, Exp.fit_mean(10 + i * 5))  # Different service rates
        delay.set_service(jclass, Exp.fit_mean(20 + i * 10))
    
    # Routing
    P = model_pop.init_routing_matrix()
    for jclass in jobclasses:
        P.add_route(jclass, queue, delay, 1.0)
        P.add_route(jclass, delay, queue, 1.0)
    
    model_pop.link(P)
    return model_pop

# Test different population mixes
population_scenarios = [
    ([5, 5], "Balanced"),
    ([10, 2], "Class1 Heavy"),
    ([2, 10], "Class2 Heavy"),
    ([8, 8], "High Load")
]

print("Population Mix | Total Pop | System Performance")
print("-" * 50)

for populations, scenario_name in population_scenarios:
    try:
        model_scenario = create_cqn_with_populations(populations)
        solver_scenario = MVA(model_scenario)
        avg_table_scenario = solver_scenario.avg_table()
        
        total_pop = sum(populations)
        # Calculate system utilization (approximate)
        util_estimate = float(avg_table_scenario.iloc[1, 1]) if len(avg_table_scenario) > 1 else 0
        
        print(f"{populations} | {total_pop:8d}  | Util≈{util_estimate:.3f} ({scenario_name})")
    except Exception as e:
        print(f"{populations} | {sum(populations):8d}  | Error: {str(e)[:20]}...")

In [ ]:
# Analyze scaling with number of classes
print("\n=== Multi-Class Scaling Analysis ===")

def create_cqn_r_classes(num_classes, population_per_class=3):
    """Create CQN with specified number of classes"""
    model_r = Network(f'CQN-{num_classes}Classes')
    
    # Single queue + delay
    queue = Queue(model_r, 'Queue', SchedStrategy.PS)
    delay = Delay(model_r, 'Delay')
    nodes = [queue, delay]
    
    # Create classes
    jobclasses = []
    for c in range(num_classes):
        jclass = ClosedClass(model_r, f'C{c+1}', population_per_class, queue, 0)
        jobclasses.append(jclass)
        
        # Differentiated service rates
        queue_rate = 5 + c * 2  # Increasing service requirements
        delay_rate = 15 + c * 3
        
        queue.set_service(jclass, Exp.fit_mean(queue_rate))
        delay.set_service(jclass, Exp.fit_mean(delay_rate))
    
    # Serial routing for all classes
    P = model_r.init_routing_matrix()
    for jclass in jobclasses:
        P.add_route(jclass, queue, delay, 1.0)
        P.add_route(jclass, delay, queue, 1.0)
    
    model_r.link(P)
    return model_r

# Test different numbers of classes
class_counts = [1, 2, 3, 4, 5]

print("Classes | Total Jobs | Queue Utilization | System Response")
print("-" * 60)

for num_classes in class_counts:
    try:
        model_classes = create_cqn_r_classes(num_classes, population_per_class=3)
        solver_classes = MVA(model_classes)
        avg_table_classes = solver_classes.avg_table()
        
        total_jobs = num_classes * 3
        queue_util = float(avg_table_classes.iloc[1, 1]) if len(avg_table_classes) > 1 else 0
        
        print(f"   {num_classes:2d}   |     {total_jobs:2d}     |      {queue_util:.3f}       | Complex system")
    except Exception as e:
        print(f"   {num_classes:2d}   |     {num_classes*3:2d}     | Error: analysis failed")

print("\nNote: As the number of classes increases, resource contention and")
print("scheduling complexity grow, affecting overall system performance.")